In [1]:
include("../src/Julia.jl")

parseData (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
gpath = "../../graphs/ipm_grid3_8000_all/";

In [4]:
graphs = readdir(gpath);
len = length(graphs)
graphs = graphs[(ceil(Int64,len/2)+1):len]

67-element Array{ByteString,1}:
 "ipm_grid3_8000_39"
 "ipm_grid3_8000_4" 
 "ipm_grid3_8000_40"
 "ipm_grid3_8000_41"
 "ipm_grid3_8000_42"
 "ipm_grid3_8000_43"
 "ipm_grid3_8000_44"
 "ipm_grid3_8000_45"
 "ipm_grid3_8000_46"
 "ipm_grid3_8000_47"
 "ipm_grid3_8000_48"
 "ipm_grid3_8000_49"
 "ipm_grid3_8000_5" 
 ⋮                  
 "ipm_grid3_8000_89"
 "ipm_grid3_8000_9" 
 "ipm_grid3_8000_90"
 "ipm_grid3_8000_91"
 "ipm_grid3_8000_92"
 "ipm_grid3_8000_93"
 "ipm_grid3_8000_94"
 "ipm_grid3_8000_95"
 "ipm_grid3_8000_96"
 "ipm_grid3_8000_97"
 "ipm_grid3_8000_98"
 "ipm_grid3_8000_99"

In [ ]:
for cur in graphs
    path = gpath * cur * "/"
    
    try
        a = readFromFile(path * "graph.mtx");
#         x = readFromFile(path * "x.vec");
        x = zeros(a.n)
        b = readFromFile(path * "b.vec");
        
        tree = []
        for ind in 1:5
            tr = readFromFile(path * "tree$(ind).mtx");
            
            u,v,w = findnz(tr)
            for i in 1:length(w)
                w[i] = a[u[i],v[i]]
            end
            tr = sparse(u,v,w)
            
            push!(tree, tr);
        end
        
        la = lap(a);
        bdef = copy(b)
        bdef = bdef - mean(bdef);
        
        numIts = 100;
        @time for i in 1:5
            @time F = treeSolver(tree[i])
            @time F(bdef);

            # a run on standard Float64 data types
            @time myx64,dbg = pcgV(la, bdef, F, x, maxits=numIts, verbose=false);
            norm(lap(a) * myx64 - bdef) / norm(bdef)
            insert!(dbg, 1, "$(numIts) iterations")
            insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
            writeToFile(path * "_log_julia_tree$(i)_default.txt", dbg)

            # now using higer precision data types
            for hp in [64,128,256,512,1024]
                set_bigfloat_precision(hp)
                laHP = toHighPrecision(la; precision=hp)
                treeHP = toHighPrecision(tree[i]; precision=hp)
                
                xHP = zeros(BigFloat, a.n)
                
                bHP = toHighPrecision(b; precision=hp);
                bHP = bHP - mean(bHP)
                
                FHP = treeSolver(treeHP)

                @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=false);
                insert!(dbg, 1, "$(numIts) iterations")
                insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
                writeToFile(path * "_log_julia_tree$(i)_$(hp).txt", dbg)
            end
        end
            
        println("finished running on ", cur, "\n")
    catch
        println("skipping ", cur, "\n")
    end
end

  0.709923 seconds (640.81 k allocations: 29.208 MB, 0.82% gc time)
  0.215695 seconds (247.50 k allocations: 6.749 MB, 2.57% gc time)
  1.301020 seconds (15.84 M allocations: 389.196 MB, 3.02% gc time)
